We have made predictions in the way of interpolating by now. Then second thing we want to do it to transform all these category variables in to frequency.(simple one-hot is not very suitable to our data) To be specifically, we'll group our dataset by 'group_1', then calculate frequency of all within single group.(number of 'type' / length of this group).  
On the other hand, keep all value type is not a good choice. It will decrease speed and require more memory with low effect on our prediction. When we want a particular type only take up a very limited space in the whole dataset. We will drop the entire column.  
At end this step, we should a get a dataframe containing difference between groups. Then, we can mutate other features.(date-related information in different groups, char_38 in different groups, act and people number in different groups)

In [1]:
import pandas as pd
import numpy as np
from operator import itemgetter
from itertools import product
from sklearn.metrics import roc_auc_score

In [2]:
train = pd.read_csv("leak_train.csv")
test = pd.read_csv("leak_test.csv")
data = train.append(test)

In [3]:
#When apply frequency method, avoid being affected by repeated observation. 
names = []
for name in data.columns:
    if name != 'act_activity_id':
        names.append(name)
#unique_train contrains all valid information
unique_data = data[~data.duplicated(subset = names)]

In [4]:
#We transform category variable via group_1 and some group only occured in test set. So we don't know  
#the actual return value these groups. We want to extract these group names first.
test_groups = test.ppl_group_1[~test.ppl_group_1.isin(train.ppl_group_1.unique())].unique()

In [5]:
#On the forum, they mentioned a important point that we should mutate more features within same group
#After we add date-related feature, it's time to do it.
mutate_features = {}
mutate_features['ppl_group_1'] = []
mutate_features['min_group_pdate'] = []
mutate_features['max_group_pdate'] = []
mutate_features['min_group_adate'] = []
mutate_features['max_group_adate'] = []
mutate_features['group_adate_range'] = []
mutate_features['group_ppl_number'] = []
mutate_features['group_act_number'] = []
mutate_features['group_adate_number'] = []

for g in data.groupby('ppl_group_1'):
    mutate_features['ppl_group_1'] .append( g[0] )
    mutate_features['min_group_adate'].append( g[1].act_date_diff.min())
    mutate_features['max_group_adate'].append( g[1].act_date_diff.max())
    mutate_features['min_group_pdate'].append( g[1].ppl_date_diff.min())
    mutate_features['max_group_pdate'].append( g[1].ppl_date_diff.max())
    mutate_features['group_adate_range'].append( g[1].act_date_diff.max() - g[1].act_date_diff.min())
    mutate_features['group_ppl_number'].append(len(g[1]['people_id'].unique()))
    mutate_features['group_act_number'].append(len(g[1]['act_activity_id'].unique()))
    mutate_features['group_adate_number'].append(len(g[1]['act_date'].unique()))
    
group_feature = pd.DataFrame(mutate_features)
#not necessary to merge
#data = pd.merge(data, ppl_in_group, on='ppl_group_1', how = 'left')


In [6]:
#Then we want to transform these category variables to the form of frequency(general onehot aren't suitable to this)
#frequency is very sensitive to repeated observation.



#get all category variables' name
names = []
for i in data.columns:
    if 'char' in i and i != 'ppl_char_38' :
        names.append(i)
    elif i == 'act_activity_category':
        names.append(i)

In [7]:
#we calculate type frequency within each group for each category variables
#create a function to do that:
def cal_freq(data, key):
    freq = {}
    #Get allunique possible value in this feature
    #col_names =  ['ppl_group_1'] + ('freq_' + data['ppl_char_5'].unique()).tolist()
    col_names = ['ppl_group_1'] + data[key].unique().tolist()
    a = data[key].unique().tolist()
    for n in col_names:
        freq[n] = []
    for g in data.groupby('ppl_group_1'):
        freq['ppl_group_1'].append(g[0])
        #get count and corresponding feature name
        c = a[:]
        value_count = g[1][key].value_counts()
        for tp in value_count.index:
            freq[tp].append(value_count[tp] / float(len(g[1][key])))
            c.remove(tp)
        for n in c:
            freq[n].append(0)
    freq = pd.DataFrame(freq)
    a = []
    for i in freq.columns:
        if i != 'ppl_group_1':
            if freq[i].mean > 0.001:
                i = key + "_"+str(i)
                a.append(i)
            else:
                freq = freq.drop(i)
        else:
            a.append(i)
    freq.columns = a
    return freq

In [8]:
for key in names:
    freq = cal_freq(unique_data, key)
    group_feature = pd.merge(group_feature, freq, on= 'ppl_group_1', how = 'left')
    del freq

In [9]:
group_feature.to_csv('group_freq.csv', index = False)

In [5]:
group_feature = pd.read_csv('group_freq.csv')

In [6]:
#Then deal with char_38 which is continues variable
group_char_38 = {}
group_char_38['char_38_mean'] = []
group_char_38['char_38_median'] = []
group_char_38['char_38_var'] = []
group_char_38['ppl_group_1'] = []

#assign value
for g in unique_data.groupby('ppl_group_1'):
    group_char_38['ppl_group_1'].append(g[0])
    group_char_38['char_38_mean'].append(g[1]['ppl_char_38'].mean())
    group_char_38['char_38_median'].append(g[1]['ppl_char_38'].median())
    if len(g[1]) == 1:
        group_char_38['char_38_var'].append(0)
    else:
        group_char_38['char_38_var'].append(g[1]['ppl_char_38'].var())
group_char_38 = pd.DataFrame(group_char_38)
group_feature = pd.merge(group_feature,group_char_38 , on ='ppl_group_1', how = 'left')
del group_char_38

In [7]:
group_train =group_feature[~group_feature.ppl_group_1.isin(test_groups)]
group_test =group_feature[group_feature.ppl_group_1.isin(test_groups)]

In [8]:
#We have mutated all features we need. Then we'll divide this group dataframe into train and test. 
#We can't assign outcome directly for train set, since some group may have different value
#But we can transform to all0, all1 and mixed value for all groups
group_outcome = {}
group_outcome['ppl_group_1'] = []
group_outcome['outcome_type'] = []

for g in unique_data.groupby('ppl_group_1'):
    group_outcome['ppl_group_1'].append(g[0])
    m = g[1].outcome.mean()
    if m == 1:
        group_outcome['outcome_type'].append(1)
    elif m == 0:
        group_outcome['outcome_type'].append(0)
    else:
        group_outcome['outcome_type'].append(2)
group_outcome = pd.DataFrame(group_outcome)
group_train = pd.merge(group_train, group_outcome, on = 'ppl_group_1', how = 'left')
del group_outcome

In [9]:
#By now, we have constructed train and test set. 
#Then create cv set
train_y = group_train['outcome_type']
train_X = group_train.drop('outcome_type', axis = 1)

from sklearn.cross_validation import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(train_y, 5, test_size=0.2, random_state=42)

cv_train_X, cv_train_y = {},{}
cv_test_X, cv_test_y = {},{}
cv_test_index = {}

i = 0
for train_index,test_index in sss:
    cv_train_X[i], cv_train_y[i] = train_X.iloc[train_index], train_y[train_index]
    cv_test_X[i], cv_test_y[i] = train_X.iloc[test_index], train_y[test_index]
    cv_test_index[i] = test_index
    i += 1

In [26]:
#implement classifier on dataset, 5 classifer for 5 split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
rf = {}
train_pred1 = []
for i in range(len(cv_train_X)):
    rf[i] = RandomForestClassifier(warm_start = True, random_state=42) #train same classifier multiple times
    for number in range(10,200, 20):
        rf[i].set_params(n_estimators = number)
        rf[i].fit(cv_train_X[i], cv_train_y[i])
    pred = rf[i].predict_proba(cv_test_X[i])
    print log_loss(cv_test_y[i], pred)
    pred = pd.DataFrame(pred, columns=['all0_pro', 'all1_pro', 'mixed_pro'])
    pred['ppl_group_1'] = train_X.iloc[cv_test_index[i]]['ppl_group_1']
    train_pred1.append(pred)

train_pred1 = pd.concat(train_pred1)

0.813095374667
0.80911067243
0.816732802912
0.818006865455
0.811422173413


In [27]:
rf = {}
train_pred2 = []
for i in range(len(cv_train_X)):
    rf[i] = RandomForestClassifier(warm_start = True, min_samples_split=8,min_samples_leaf=2,n_jobs=8, random_state=42) #train same classifier multiple times
    for number in range(10,200, 20):
        rf[i].set_params(n_estimators = number)
        rf[i].fit(cv_train_X[i], cv_train_y[i])
    pred = rf[i].predict_proba(cv_test_X[i])
    print log_loss(cv_test_y[i], pred)
    pred = pd.DataFrame(pred, columns=['all0_pro', 'all1_pro', 'mixed_pro'])
    pred['ppl_group_1'] = train_X.iloc[cv_test_index[i]]['ppl_group_1']
    train_pred2.append(pred)

train_pred2 = pd.concat(train_pred2)

0.810516282386
0.81603166933
0.816075666662
0.81413578847
0.816371982218


In [28]:
rf = {}
train_pred3 = []
for i in range(len(cv_train_X)):
    rf[i] = RandomForestClassifier(warm_start = True, min_samples_split=12,min_samples_leaf=2,n_jobs=8, random_state=42) #train same classifier multiple times
    for number in range(10,200, 20):
        rf[i].set_params(n_estimators = number)
        rf[i].fit(cv_train_X[i], cv_train_y[i])
    pred = rf[i].predict_proba(cv_test_X[i])
    print log_loss(cv_test_y[i], pred)
    pred = pd.DataFrame(pred, columns=['all0_pro', 'all1_pro', 'mixed_pro'])
    pred['ppl_group_1'] = train_X.iloc[cv_test_index[i]]['ppl_group_1']
    train_pred3.append(pred)

train_pred3 = pd.concat(train_pred3)

0.81075512951
0.815339952769
0.818442570608
0.814008939503
0.81546569301


In [10]:
import xgboost as xgb
from sklearn.metrics import log_loss

In [11]:
xg_train = {}
xg_test = {}
for i in range(5):
    xg_train[i] = xgb.DMatrix( cv_train_X[i], label= cv_train_y[i])
    xg_test[i] = xgb.DMatrix(cv_test_X[i], label= cv_test_y[i])

In [12]:
# setup parameters for xgboost

param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softprob'
# scale weight of positive examples
param['eta'] = 0.01
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 3
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['eval_metric'] = 'mlogloss'

bst = {}
pred = {}
for f in range(5):
    watchlist = [ (xg_train[f],'train'), (xg_test[f], 'test') ]
    num_round = 2500
    bst[f] = xgb.train(param, xg_train[f], num_round, watchlist , verbose_eval = 10, early_stopping_rounds=30)

[0]	train-mlogloss:1.09501	test-mlogloss:1.09514
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 30 rounds.
[10]	train-mlogloss:1.0608	test-mlogloss:1.06195
[20]	train-mlogloss:1.03055	test-mlogloss:1.03274
[30]	train-mlogloss:1.00409	test-mlogloss:1.00731
[40]	train-mlogloss:0.98031	test-mlogloss:0.984481
[50]	train-mlogloss:0.958882	test-mlogloss:0.964266
[60]	train-mlogloss:0.93996	test-mlogloss:0.946385
[70]	train-mlogloss:0.921829	test-mlogloss:0.929494
[80]	train-mlogloss:0.905611	test-mlogloss:0.91443
[90]	train-mlogloss:0.890927	test-mlogloss:0.900988
[100]	train-mlogloss:0.8776	test-mlogloss:0.888691
[110]	train-mlogloss:0.865203	test-mlogloss:0.877586
[120]	train-mlogloss:0.854466	test-mlogloss:0.868052
[130]	train-mlogloss:0.844143	test-mlogloss:0.858922
[140]	train-mlogloss:0.834563	test-mlogloss:0.850639
[150]	train-mlogloss:0.825695	test-mlogloss:0.842848
[160]	train-mlogloss:0.817

In [13]:
xgb_cv_pred = {}


from sklearn.metrics import log_loss

xgb_cv_preds = []
df_preds = []
for f in range(5):
    #preds = et[f].predict_proba(X_test[f])
    xgb_cv_pred[f] = bst[f].predict( xg_test[f], ntree_limit=bst[f].best_ntree_limit)
    df_preds.append(pd.DataFrame(xgb_cv_pred[f]))
    df_preds[-1].columns = ['gp_all0', 'gp_all1', 'gp_mixed']
    df_preds[-1]['ppl_group_1'] = train_X.iloc[cv_test_index[f]].ppl_group_1.values

df_preds_all = pd.concat(df_preds)

#train_preds = np.array(df_preds_all.sort_values('ppl_group_1').drop('ppl_group_1', axis=1))

#log_loss(gtrain.otype, train_preds)

In [14]:
gtesta = group_test.copy()
xgb_output = xgb.DMatrix(gtesta)

In [23]:
test_preds = []
for f in range(5):
    test_preds.append(bst[f].predict(xgb_output, ntree_limit = bst[f].best_ntree_limit))
    
preds_comb = np.mean([test_preds[j] for j in range(1,5)], axis = 0)

df_test_preds = pd.DataFrame(preds_comb)
df_test_preds.columns = ['gp_all0', 'gp_all1', 'gp_mixed']
df_test_preds['ppl_group_1'] = gtesta.ppl_group_1.values

In [26]:
preds_out = pd.concat([df_test_preds, df_preds_all])

preds_out.to_csv('group_pred.csv', index = False)

In [18]:
preds_out = preds_out.drop('ppl_group_1', axis = 1)
preds_out.columns = ['gp_all0', 'gp_all1', 'gp_mixed', 'ppl_group_1']

In [29]:
preds_out.to_csv('group_pred.csv', index = False)

In [24]:
preds_out = pd.concat([df_test_preds, df_preds_all])

In [25]:
sum(preds_out['ppl_group_1'].isnull())

0

In [27]:
len(preds_out) == len(df_preds_all) +len(df_test_preds)

True

In [28]:
preds_out

,gp_all0,gp_all1,gp_mixed,ppl_group_1
0,0.156040,0.842834,0.001126,15
1,0.093303,0.633365,0.273332,18
2,0.278425,0.460493,0.261082,53
3,0.244403,0.590451,0.165146,85
4,0.850534,0.120584,0.028882,99
5,0.237691,0.760850,0.001458,126
6,0.981711,0.008074,0.010215,134
7,0.262060,0.434307,0.303633,182
8,0.158581,0.720605,0.120815,235
9,0.465105,0.278054,0.256841,293
